# 8. Multi-GPU 학습

> PyTorch에서 Multi GPU를 사용하기 위해 딥러닝 모델을 병렬화 하는 Model Parallel의 개념과 데이터 로딩을 병렬화하는 Data Parallel의 개념을 학습합니다. 이를 통해 다중 GPU 환경에서 딥러닝을 학습할 때에 효율적으로 하드웨어를 사용할 수 있도록 하고, 더 나아가 딥러닝 학습 시에 GPU가 동작하는 프로세스에 대한 개념을 익힙니다.

<br>

## Reference

- [Pytorch Lightning Multi GPU 학습](https://pytorch-lightning.readthedocs.io/en/stable/advanced/multi_gpu.html)
- [DDP 튜토리얼](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)

<br>

## 8.1 Multi-GPU

- 어떻게 GPU 를 다룰 것인가

<br>

## 8.2 개념 정리

### 8.2.1 기본 용어

- Single vs. Multi
- GPU vs. Node(or System)
  - Node(or System) : 1대의 컴퓨터를 의미
- Single Node Single GPU
  - 1대의 컴퓨터에 있는 1개의 GPU 사용
- Single Node Multi GPU
  - 1대의 컴퓨터에 있는 여러 개의 GPU 사용
  - 대부분의 상황이 여기에 해당된다.
- Multi Node Multi GPU
  - 여러 대의 컴퓨터에 있는 여러 개의 GPU 사용
  - 구성하는 방법이 만만치 않다.

<br>

### 8.2.2 TensorRT

- GPU 를 쉽게 사용할 수 있게 해주는 도구

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1-u3T12fo8nxwkR47jypb9Y5yn79zZMmk' width=800/>

<br>

### 8.2.3 다중 GPU에 학습을 분산하는 두 가지 방법

- 모델을 나누기 (Model parallel)
- 데이터를 나누기 (Data parallel)

<br>

## 8.3 Model parallel

- 모델을 나누는 것은 생각보다 예전부터 썼음 (alexnet)
- 모델의 병목, 파이프라인의 어려움 등으로 인해 모델 병렬화는 고난이도 과제
- 흔하게 많이 쓰이진 않음

<br>

### 8.3.1 AlexNet

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1as7MMPKtk6jBR5zuH0_fPjujwS5mw4RJ' width=800/>

- 교차되는 부분
  - `to(...)`
  - GPU 간의 병렬적인 처리를 지원하기 위함

<br>

### 8.3.2 모델 병렬화 시 발생하는 문제점

- 병렬화를 잘못하면 빈 공간이 생겨 병렬화의 효과를 얻을 수 없다.
- 위 그림이 잘못된 예시, 아래 그림이 제대로된 예시
- 제대로 병렬화를 하려면 아래 그림과 같이 파이프라인이 겹쳐야 한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1DUW-y571QkFkL2RP--HclG4s0JgLsTEV' width=800/>

- 출처 : http://www.idris.fr/eng/ia/model-parallelism-pytorch-eng.html

<br>

### 8.3.3 Model parallel 구현 코드

```python
class ModelParallelResNet50(ResNet):
    def __init__(self, *args, **kwargs):
        super(ModelParallelResNet50, self).__init__(
            Bottelneck, [3, 4, 6, 3], num_classes=num_classes, *args, **kwargs)

        # 첫 번째 모델을 cuda 0 에 할당
        self.seq1 = nn.Sequential(
            self.conv1, self.bn1, self.relu, self.maxpool, self.layer1, self.layer2
        ).to('cuda:0')

        # 두 번째 모델을 cuda 1 에 할당
        self.seq2 = nn.Sequential(
            self.layer3, self.layer4, selfl.avgpool
        ).to('cuda:1')
        self.fc.to('cuda:1')

    def forward(self, x):
        # cuda 1 으로 cuda 0 에 있던 모델을 복사하여 두 모델 연결하기
        x = self.seq2(self.seq1(x).to('cuda:1'))
        return self.fc(x.view(x.size(0), -1))
```

- 하지만 위와 같이 구현하면 데이터 병목현상이 발생할 수 있다.

<br>

## 8.4 Data parallel

- 데이터를 나눠 GPU에 할당후 결과의 평균을 취하는 방법
- minibatch 수식과 유사한데 한번에 여러 GPU에서 수행

<br>

### 8.4.1 수행 절차

- 각각의 GPU에 데이터를 나눔
- 각각의 GPU에 모델을 복제
- 각각의 GPU에서 모델 학습
- 연산한 결과값을 하나의 GPU로 모음
- 모여진 GPU에서 각각의 loss 값을 한 번에 계산
  - global interpretor lock (파이썬 멀티프로세싱 제약사항)
- 구해진 각각의 gradient 를 개별 GPU로 나눔
- 각각의 GPU들이 backward 를 수행
- 업데이트된 weight 값을 하나의 GPU로 모아 평균을 계산하여 최종적인 학습이 진행됨

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1D_uswzu6zQcs_4-FDQPpwdb0VyWtq3uf' width=900/>

- 출처: https://bit.ly/37usURV

<br>

- 정보가 모아지는 GPU가 과부하가 걸려 문제가 생길 수 있음 (8.4.2.1 참고)

<br>

### 8.4.2 두 가지 방식의 Data parallel

- PyTorch에서는 아래 두 가지 방식을 제공
  - `DataParallel`
  - `DistributedDataParallel`

<br>

#### 8.4.2.1 `DataParallel`

- 단순히 데이터를 분배한후 평균을 취함
- GPU 사용 불균형 문제 발생
- Batch 사이즈 감소 (한 GPU가 병목)
- GIL (Global Interpretor Lock)

<br>

#### 8.4.2.2 `DistributedDataParallel`

- 각 CPU마다 process 생성하여 개별 GPU에 할당
- 기본적으로 DataParallel로 하나 개별적으로 연산의 평균을 냄

<br>

- loss를 하나의 GPU로 모으는 작업이 없음
- 각각의 GPU 가 해당 작업을 수행하고 평균치를 반영하는 방식

<br>

### 8.4.3 `DataParallel` 구현 코드

```python
parallel_model = torch.nn.DataParallel(model) # Encapsulate the model
# 이게 전부 ... (DataParallel() 만 붙여주면 된다.)

predictions = parallel_model(inputs) # Forward pass on multi-GPUs
loss = loss_function(predictions, labels) # Compute loss function
loss.mean().backward() # Average GPU-losses + backward pass
optimizer.step() # Optimizer step
predictions = parallel_model(inputs) # Forward pass with new parameters
```

- 출처: https://bit.ly/37usURV

<br>

### 8.4.4 `DistributedDataParallel` 구현 코드

```python
# Sampler 를 사용
#  - Sampler : 인덱스를 어떻게 선택하느냐의 문제
train_sampler = torch.utils.data.distributed.DistributedSampler(train_data) 
shuffle = False
pin_memory = True

trainloader = torch.utils.data.DataLoader(
    train_data, batch_size=20, shuffle=True
    pin_memory=pin_memory, # pin_memory 를 True 로 지정
    num_workers=3, # GPU 갯수 x 4 를 추천
    shuffle=shuffle,
    sampler=train_sampler # dataloader 에 sampler 를 전달
)
```

- `pin_memory=True`

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=15M1_wNJ2rxfH2GWQqYZr6kzzb_WCZpjY' width=500/>

- pin memory
  - DRAM(메모리)에 데이터를 바로바로 올릴 수 있도록 절차를 간단하게 하여 데이터를 저장하는 방식

<br>

- Python 멀티프로세싱 코드

```python
def main():
    n_gpus = torch.cuda.device_count() # gpu 갯수
    torch.multiprocessing.spawn(main_worker, nprocs=n_gpus, args=(n_gpus, ))

def main_worker(gpu, n_gpus):
    image_size = 224
    batch_size = 512
    num_worker = 8
    epochs = ...

    # batch_size 와 num_worker 를 gpu 갯수만큼 잘라줘야 한다.
    batch_size = int(batch_size / n_gpus)
    num_worker = int(num_worker / n_gpus)

    # 멀티프로세싱 통신 규약 정의
    #  - 각각의 멀티프로세싱을 구성하는 각각의 프로세스들이 데이터를 주고 받기 위한 통신 규약 정의
    torch.distributed.init_process_group(
    backend='nccl’, init_method='tcp://127.0.0.1:2568’, world_size=n_gpus,  rank=gpu)

    model = MODEL

    torch.cuda.set_device(gpu)
    model = model.cuda(gpu)

    # Distributed dataparallel 정의
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[gpu])

from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    # Python 의 멀티프로세싱 코드
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3])) # 5개의 pool 에서 1^2, 2^2, 3^3 를 구해준다.
```

- 출처: https://blog.si-analytics.ai/12